# 如何处理缺失值

现实和教程最大的区别之一是，现实的数据集是混乱不堪的，数据科学家90%的时间都花在数据预处理上，其中就包括处理缺失值(missing values)。

Python做数据科学项目时通常用Pandas存储数据，所以我们重点讨论如何用Pandas处理缺失值。

## 1. 如何表示缺失值

用python处理数据主要通过numpy和pandas实现，所以首先要理解它们是如何表示缺失值的。

In [1]:
import pandas as pd
import numpy as np

Python用None表示缺失值，如果创建一个包含None的列表，是无法直接进行数值运算的，调用mean,sum等数值计算函数会引发'TypeError'.

In [2]:
l = [1, 2, None, 4]
l

[1, 2, None, 4]

创建一个包含None的numpy数组。

In [3]:
nums = np.array([1, 2, None, 4])
nums

array([1, 2, None, 4], dtype=object)

dtype=object，当数组包含None，numpy将数组的类型推断为'object'，即包含原生的python对象，同样无法进行数值运算。

In [21]:
try:
    np.sum(nums)
except Exception as e:
    print(e)

unsupported operand type(s) for +: 'int' and 'NoneType'


Numpy提供了一种特殊的数据类型，np.nan，表示缺失值，就本质上而言，它是浮点值。

In [5]:
arr = np.array([1, 2, np.nan, 4])
arr

array([ 1.,  2., nan,  4.])

包含np.nan的数组可以进行数值运算而不引发类型异常。

需要明确一点，涉及np.nan的运算结果总是np.nan，忽略缺失值才能得到正确的结果，numpy提供了一系列针对缺失值的数值运算函数。

In [6]:
# 结果是np.nan
print(np.mean(arr), np.sum(arr), np.min(arr), np.max(arr))

# 计算时剔除np.nan，获得正确结果
print(np.nanmean(arr), np.nansum(arr), np.nanmin(arr), np.nanmax(arr))

nan nan nan nan
2.3333333333333335 7.0 1.0 4.0


在pandas中可以同时使用None和np.nan表示缺失值。

创建pandas数组时，如果数组中包含None，会自动转换为np.nan，不仅如此，其它整数也被自动转化为浮点值。

In [7]:
ser = pd.Series([1, 2, None, np.nan])
ser

0    1.0
1    2.0
2    NaN
3    NaN
dtype: float64

## 2. 处理缺失值

从上面的讨论可知，pandas使用None和np.nan表示缺失值，pd.Series和pd.DataFrame均提供了几个常用方法来侦测，剔除和填充缺失值，它们分别是：

* isnull()
* notnull()
* dropna()
* fillna()

In [8]:
# 先创建一个包含缺失值的series
ser = pd.Series([1, None, 3, np.nan, 5])
ser

0    1.0
1    NaN
2    3.0
3    NaN
4    5.0
dtype: float64

### 2.1 侦测缺失值

isnull()返回布尔数组，如果元素是None或np.nan，返回True，否则返回False。

In [9]:
ser.isnull()

0    False
1     True
2    False
3     True
4    False
dtype: bool

notnull()是isnull()的逆运算，可以用于筛选非缺失的数据。

In [10]:
ser[ser.notnull()]

0    1.0
2    3.0
4    5.0
dtype: float64

### 2.2 剔除缺失值

在剔除缺失值之前，最好先理解缺失值产生的原因。导致数据缺失的原因有很多，例如记录员忘记录入数据，或由于条件限制无法获得真实的观测值等。

剔除缺失值可能是最简单的处理方法，但只有在数据集足够大，缺失数据的比例在可控范围内的时候才推荐使用。

In [11]:
# 先创建包含缺失值的数据框
arr = np.array([
    [1, 2, np.nan],
    [np.nan, np.nan, np.nan],
    [np.nan, 4, 5],
    [6, 7, 8]
])
df = pd.DataFrame(arr, columns=["A", "B", "C"])
df

,A,B,C
0,1.0,2.0,NaN
1,NaN,NaN,NaN
2,NaN,4.0,5.0
3,6.0,7.0,8.0


dropna()默认剔除包含任意缺失值的行，通过指定axis=1可以对列进行操作。

In [14]:
# 剔除包含任意缺失值的行
df.dropna()

# 剔除包含任意缺失值的列
# df.dropna(axis=1)

,A,B,C
3,6.0,7.0,8.0


指定参数how="all"，剔除全是缺失值的行。

In [15]:
df.dropna(how="all")

,A,B,C
0,1.0,2.0,NaN
2,NaN,4.0,5.0
3,6.0,7.0,8.0


如果想保留缺失值较少的行/列，可以指定'thresh'。

In [16]:
# 当非缺失值的数量大于等于thresh，保留该行
df.dropna(thresh=2)

,A,B,C
0,1.0,2.0,NaN
2,NaN,4.0,5.0
3,6.0,7.0,8.0


### 2.3 填充缺失值

有时候我们想填充而不是剔除缺失值，在调用fillna()之前，首先要明确用什么数据进行填充，下面介绍几种常用方法。

最简单的方法是用单一值填充，例如0，但这种方法的实用性不强。

In [17]:
df.fillna(0)

,A,B,C
0,1.0,2.0,0.0
1,0.0,0.0,0.0
2,0.0,4.0,5.0
3,6.0,7.0,8.0


向前填充或向后填充，即用前面(后面)的观测值填充后面(前面)的缺失值。

In [18]:
# ffill表示forward fill, 向前填充
df.fillna(method="ffill")

,A,B,C
0,1.0,2.0,NaN
1,1.0,2.0,NaN
2,1.0,4.0,5.0
3,6.0,7.0,8.0


In [19]:
# bfill表示backward fill, 向后填充
df.fillna(method="bfill")

,A,B,C
0,1.0,2.0,5.0
1,6.0,4.0,5.0
2,6.0,4.0,5.0
3,6.0,7.0,8.0


用变量的推断值填充，这可能是最实用的方法，例如计算数值特征的均值，中位数或众数，用这些统计量填充缺失值。

In [20]:
# 计算每一列的均值，用均值填充
df.fillna(df.mean())

,A,B,C
0,1.0,2.000000,6.5
1,3.5,4.333333,6.5
2,3.5,4.000000,5.0
3,6.0,7.000000,8.0
